In [10]:
import pandas as pd
import numpy as np
import mlflow
import math
import random
import datetime as dt

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, TimeDistributed, LSTM, Add,Concatenate
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import pickle

In [11]:
df = pd.read_csv("Daily Stock Data.csv")
df.head()

,CompanyID,Date,Open,High,Low,Close,Volume,Change,Previous Close,Previous Close Filled,True Range,OBV,OBV_MovingAvg_14,OBV_Oscillator
0,1,10/28/2019 0:00,107.99,108.46,106.45,106.60,7000000,-0.0055,NaN,106.6,2.01,-7000000,-7000000,0
1,1,10/29/2019 0:00,106.84,107.02,104.69,105.00,5420000,-0.0150,106.6,106.6,2.33,-5420000,-5420000,0
2,1,10/30/2019 0:00,105.29,106.60,103.96,106.50,4200000,0.0143,106.6,106.6,2.64,4200000,4200000,0
3,1,10/31/2019 0:00,106.47,106.50,103.26,104.10,7200000,-0.0225,106.6,106.6,3.34,-7200000,-7200000,0
4,1,11/1/2019 0:00,104.70,105.30,103.93,104.98,5490000,0.0085,106.6,106.6,2.67,5490000,5490000,0


In [23]:
df.shape

(12562, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12562 entries, 0 to 12561
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CompanyID              12562 non-null  int64  
 1   Date                   12562 non-null  object 
 2   Open                   12562 non-null  float64
 3   High                   12562 non-null  float64
 4   Low                    12562 non-null  float64
 5   Close                  12562 non-null  float64
 6   Volume                 12562 non-null  int64  
 7   Change                 12562 non-null  float64
 8   Previous Close         12552 non-null  float64
 9   Previous Close Filled  12562 non-null  float64
 10  True Range             12562 non-null  float64
 11  OBV                    12562 non-null  int64  
 12  OBV_MovingAvg_14       12562 non-null  int64  
 13  OBV_Oscillator         12562 non-null  int64  
dtypes: float64(8), int64(5), object(1)
memory usage: 1.3+ 

In [12]:
def specific_data_company(id_company):
  company_data = df[df["CompanyID"] == id_company]
  return company_data

In [13]:
AAPL = specific_data_company(1)
AAPL.shape

(1258, 14)

In [14]:
AAPL['Date'] = pd.to_datetime(AAPL['Date'])

C:\Users\DELL inspiron\AppData\Local\Temp\ipykernel_776\3014602779.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AAPL['Date'] = pd.to_datetime(AAPL['Date'])


In [6]:
AAPL.head()

,CompanyID,Date,Open,High,Low,Close,Volume,Change,Previous Close,Previous Close Filled,True Range,OBV,OBV_MovingAvg_14,OBV_Oscillator
0,1,2019-10-28,107.99,108.46,106.45,106.60,7000000,-0.0055,NaN,106.6,2.01,-7000000,-7000000,0
1,1,2019-10-29,106.84,107.02,104.69,105.00,5420000,-0.0150,106.6,106.6,2.33,-5420000,-5420000,0
2,1,2019-10-30,105.29,106.60,103.96,106.50,4200000,0.0143,106.6,106.6,2.64,4200000,4200000,0
3,1,2019-10-31,106.47,106.50,103.26,104.10,7200000,-0.0225,106.6,106.6,3.34,-7200000,-7200000,0
4,1,2019-11-01,104.70,105.30,103.93,104.98,5490000,0.0085,106.6,106.6,2.67,5490000,5490000,0


In [29]:
AAPL.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1258 entries, 0 to 1257
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CompanyID              1258 non-null   int64         
 1   Date                   1258 non-null   datetime64[ns]
 2   Open                   1258 non-null   float64       
 3   High                   1258 non-null   float64       
 4   Low                    1258 non-null   float64       
 5   Close                  1258 non-null   float64       
 6   Volume                 1258 non-null   int64         
 7   Change                 1258 non-null   float64       
 8   Previous Close         1257 non-null   float64       
 9   Previous Close Filled  1258 non-null   float64       
 10  True Range             1258 non-null   float64       
 11  OBV                    1258 non-null   int64         
 12  OBV_MovingAvg_14       1258 non-null   int64         
 13  OBV_Osci

In [15]:
# Get the complete date range for each CompanyID
def complete_date_range(group):
    full_date_range = pd.date_range(start=group["Date"].min(), end=group["Date"].max())
    return pd.DataFrame({"Date": full_date_range, "CompanyID": group["CompanyID"].iloc[0]})

# Apply function to get the full dataset with all dates
AAPL_full = pd.concat([complete_date_range(group) for _, group in AAPL.groupby("CompanyID")])

# Merge the full date range with the original data
AAPL_full = AAPL_full.merge(AAPL, on=["CompanyID", "Date"], how="left")

# Fill missing values
# Forward fill numerical data
AAPL_full.sort_values(by=["CompanyID", "Date"], inplace=True)

AAPL_full["Close"] = AAPL_full["Close"].ffill()
AAPL_full["True Range"] = AAPL_full["True Range"].ffill()
AAPL_full["OBV_MovingAvg_14"] = AAPL_full["OBV_MovingAvg_14"].ffill()


In [16]:
AAPL_full = AAPL_full[["Date", "Close", "True Range", "OBV_MovingAvg_14"]]
AAPL_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1825 non-null   datetime64[ns]
 1   Close             1825 non-null   float64       
 2   True Range        1825 non-null   float64       
 3   OBV_MovingAvg_14  1825 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 57.2 KB


In [8]:
AAPL_full = AAPL_full.sort_values("Date")
AAPL_full.tail(2)

,Date,Close,True Range,OBV_MovingAvg_14
1823,2024-10-24,81.39,227.61,7400000.0
1824,2024-10-25,81.70,226.83,7200000.0


In [17]:
scaler = MinMaxScaler()
features = ["Close", "True Range", "OBV_MovingAvg_14"]
AAPL_Scaler = scaler.fit_transform(AAPL_full[features])
AAPL_Scaled = pd.DataFrame(AAPL_Scaler, columns=features)
AAPL_Scaled.head(1)

,Close,True Range,OBV_MovingAvg_14
0,0.21775,0.003422,0.680208


In [10]:
print(AAPL_Scaled.isna().sum())

Close               0
True Range          0
OBV_MovingAvg_14    0
dtype: int64


In [18]:
def create_sequences(data, time_steps=5):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])  # All features
        y.append(data[i + time_steps, 0])     # Target is 'Close'
    return np.array(X), np.array(y)

time_steps = 5
X, y = create_sequences(AAPL_Scaled.values, time_steps)

print(X[0])
print(y[0])

[[0.21775006 0.00342213 0.68020839]
 [0.21155187 0.00466654 0.68852286]
 [0.21736267 0.00587206 0.73914645]
 [0.20806539 0.00859421 0.67915592]
 [0.21147439 0.00598872 0.74593485]]
0.2114743937398311


In [19]:
# Split data into training and testing sets
def split_data(X, y):
  train_size = int(len(X) * 0.8)
  X_train, X_test = X[:train_size], X[train_size:]
  y_train, y_test = y[:train_size], y[train_size:]
  print(y_test.shape)
  print(X_train[0])
  return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_data(X, y)
print(X_test.shape)

(364,)
[[0.21775006 0.00342213 0.68020839]
 [0.21155187 0.00466654 0.68852286]
 [0.21736267 0.00587206 0.73914645]
 [0.20806539 0.00859421 0.67915592]
 [0.21147439 0.00598872 0.74593485]]
(364, 5, 3)


In [20]:
# Define the Transformer model
def build_transformer_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=8, key_dim=128)(inputs, inputs)
    x = layers.LayerNormalization()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1)(x)  # Predict stock price
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model_transform = build_transformer_model(X_train.shape[1:])
model_transform.compile(optimizer="adam", loss="mean_squared_error")

model_transform.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 3)      │     15,363 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 5, 3)      │          6 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5, 64)     │        256 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 320)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        321 │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,946 (62.29 KB)

 Trainable params: 15,946 (62.29 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
params = {
    "count_params": model_transform.count_params()
}
params

{'count_params': 15946}

In [ ]:
model_transform.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    verbose=1
)

Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 43s 131ms/step - loss: 0.0488 - val_loss: 0.0010
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - loss: 0.0011 - val_loss: 5.2677e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 14s 142ms/step - loss: 0.0010 - val_loss: 7.8236e-05
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 20s 127ms/step - loss: 9.0704e-04 - val_loss: 6.3196e-05
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 129ms/step - loss: 9.4433e-04 - val_loss: 4.8335e-05
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 127ms/step - loss: 7.0012e-04 - val_loss: 9.0461e-05
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 22s 140ms/step - loss: 9.3923e-04 - val_loss: 6.6699e-05
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - loss: 6.7839e-04 - val_loss: 3.8218e-04
Epoch 9/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 12s 134ms/step - loss: 6.6453e-04 - val_loss: 3.0889e-04
Epoch 10/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 142ms/step - loss: 8.3507e-04 - val_loss: 2.8840e-04
Epoch 11/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 13s 141ms/step - loss: 6.47

In [16]:
# استرجاع المُحسِّن المستخدم في النموذج
optimizer_transformer = model_transform.optimizer

# عرض جميع البرامترات المُستخدمة
optimizer_transformer_config = optimizer_transformer.get_config()

# طباعة القيم
print("Optimizer Parameters:")
for key, value in optimizer_transformer_config.items():
    print(f"{key}: {value}")

Optimizer Parameters:
name: adam
learning_rate: 0.0010000000474974513
weight_decay: None
clipnorm: None
global_clipnorm: None
clipvalue: None
use_ema: False
ema_momentum: 0.99
ema_overwrite_frequency: None
loss_scale_factor: None
gradient_accumulation_steps: None
beta_1: 0.9
beta_2: 0.999
epsilon: 1e-07
amsgrad: False


In [21]:
# Predict scaled values
def evaluation(X_test, model):
  y_pred_scaled = model.predict(X_test)

  # Pad the predictions and true values to match the scaler's input shape
  y_pred_full = np.hstack((y_pred_scaled, np.zeros((len(y_pred_scaled), 2))))  # Add zeros for the other two features
  y_test_full = np.hstack((y_test.reshape(-1, 1), np.zeros((len(y_test), 2))))

  # Inverse transform
  y_pred_rescaled = scaler.inverse_transform(y_pred_full)[:, 0]  # Extract the "Close" column
  y_test_rescaled = scaler.inverse_transform(y_test_full)[:, 0]  # Extract the "Close" column

  mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
  mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)

  print(f"Mean Absolute Error (MAE): {mae:.4f}")
  print(f"Mean Squared Error (MSE): {mse:.4f}")
  # Create a DataFrame for comparison
  result_df = pd.DataFrame({
      "Close": y_test_rescaled,
      "Predicted_Close_transform": y_pred_rescaled
  })
  return result_df, mse, mae

In [ ]:
result_df, mse, mae = evaluation(X_test, model_transform)

12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step
Mean Absolute Error (MAE): 1.3452
Mean Squared Error (MSE): 3.1610


In [19]:
accurcy_transformer ={
    "MAE": mae,
    "MSE": mse
}

In [22]:
# test on data future
def predict_future(model, periods):
    scaler = MinMaxScaler()
    df = pd.read_csv("E:/Task6/Stock_data_test.csv")
    # البيانات المدخلة
    sequnce = df.loc[14:18, ["close", "true_range", "OBV_Moving14"]]
    # sequnce = np.array(sequnce).reshape(-1, 1)
    sequnce = scaler.fit_transform(sequnce)
    print(sequnce)
    predictions = []
    sequence = sequnce.copy()
    time_steps, num_features = sequence.shape  # Infer time steps and feature count

    for _ in range(periods):
        # Reshape sequence for prediction
        scaled_pred = model.predict(sequence.reshape(1, time_steps, num_features))

        # Pad scaled_pred to match feature dimensions
        padded_pred = np.hstack((scaled_pred, np.zeros((1, num_features - 1))))

        # Inverse transform the prediction
        pred = scaler.inverse_transform(padded_pred)
        predictions.append(
            pred[0][0]
        )  # Append the unscaled predicted value (first column)

        # Update the sequence by appending the prediction and removing the oldest value
        sequence = np.vstack((sequence[1:], padded_pred[0]))
    predictions_df = pd.DataFrame(
        {"Predicted Prices": predictions}
    )
 
    return predictions_df

In [21]:
pred = predict_future(model_transform, 59)

[[0.         0.98773006 0.        ]
 [0.0787234  0.57055215 0.196788  ]
 [0.68510638 1.         0.3723829 ]
 [1.         0.         0.76347328]
 [0.86808511 0.90797546 1.        ]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 680ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 802ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 684ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 769ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 810ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 866ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 601ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 

In [23]:
pred_new_data, test_new_data = [], []
for i in range(len(pred.values)):
    pred_new_data.append(pred.values[i , 0])
pred_new_data = np.array(pred_new_data)

df = pd.read_csv("E:\Task6\Stock_data_test.csv")
df = df[["close"]]

for i in range(len(df.values)-14):
    test_new_data.append(df.values[i+14 , 0])
test_new_data = np.array(test_new_data)

print(test_new_data.shape)
print(pred_new_data.shape)
test_new_data


(59,)
(59,)


<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\DELL inspiron\AppData\Local\Temp\ipykernel_5348\2430592901.py:6: SyntaxWarning: invalid escape sequence '\T'
  df = pd.read_csv("E:\Task6\Stock_data_test.csv")


array([231.78, 232.15, 235.  , 236.48, 235.86, 230.76, 230.57, 231.41,
       233.4 , 233.67, 230.1 , 225.91, 222.91, 222.01, 223.45, 222.72,
       227.48, 226.96, 224.23, 224.23, 225.12, 228.22, 225.  , 228.02,
       228.28, 229.  , 228.52, 229.87, 232.87, 235.06, 234.93, 237.33,
       239.59, 242.65, 243.01, 243.04, 242.84, 246.75, 247.77, 246.49,
       247.96, 248.13, 251.04, 253.48, 248.05, 249.79, 254.49, 255.27,
       258.2 , 259.02, 255.59, 252.2 , 250.42, 243.85, 243.36, 245.  ,
       242.21, 242.7 , 236.85])

In [25]:
  mse = mean_squared_error(test_new_data, pred_new_data)
  mae = mean_absolute_error(test_new_data, pred_new_data)

  print(f"Mean Absolute Error (MAE): {mae:.4f}")
  print(f"Mean Squared Error (MSE): {mse:.4f}")

Mean Absolute Error (MAE): 9.7328
Mean Squared Error (MSE): 147.4360


In [ ]:
accurce_transformer_newdata ={
    "MSE": mse,
    "MAE": mae
}

In [42]:
with open("apple_model_transformer.pkl", "wb") as f:
    pickle.dump(model_transform, f)  # save full model

In [26]:
# Build LSTM model
model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 3)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])
model_lstm.compile(optimizer="adam", loss="mean_squared_error")

# Train the model

model_lstm.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1)

c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 71s 210ms/step - loss: 0.0761 - val_loss: 1.6092e-04
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 34s 361ms/step - loss: 0.0036 - val_loss: 2.5712e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 60s 528ms/step - loss: 0.0033 - val_loss: 1.9501e-04
Epoch 4/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 97s 652ms/step - loss: 0.0029 - val_loss: 5.8334e-05
Epoch 5/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 62s 367ms/step - loss: 0.0026 - val_loss: 6.1763e-05
Epoch 6/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 41s 366ms/step - loss: 0.0026 - val_loss: 2.1856e-04
Epoch 7/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 38s 322ms/step - loss: 0.0027 - val_loss: 1.0025e-04
Epoch 8/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 62s 517ms/step - loss: 0.0021 - val_loss: 1.0935e-04
Epoch 9/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 79s 421ms/step - loss: 0.0025 - val_loss: 4.6054e-05
Epoch 10/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 41s 384ms/step - loss: 0.0023 - val_loss: 1.8315e-04
Epoch 11/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 40s 340ms/step - loss: 0.0021 - val_loss: 2.0733e-

In [27]:
optimizer_lstm = model_lstm.optimizer

# عرض جميع البرامترات المُستخدمة
optimizer_lstm_config = optimizer_lstm.get_config()

# طباعة القيم
print("Optimizer Parameters:")
for key, value in optimizer_lstm_config.items():
    print(f"{key}: {value}")

Optimizer Parameters:
name: adam
learning_rate: 0.0010000000474974513
weight_decay: None
clipnorm: None
global_clipnorm: None
clipvalue: None
use_ema: False
ema_momentum: 0.99
ema_overwrite_frequency: None
loss_scale_factor: None
gradient_accumulation_steps: None
beta_1: 0.9
beta_2: 0.999
epsilon: 1e-07
amsgrad: False


In [29]:
result_df, mse, mae = evaluation(X_test, model_lstm)

12/12 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step
Mean Absolute Error (MAE): 2.0489
Mean Squared Error (MSE): 6.0324


In [32]:
accurce_lstm ={
    "MSE": mse,
    "MAE": mae
}

In [30]:
result_df.head(5)

,Close,Predicted_Close_transform
0,50.39,54.006414
1,50.39,52.685319
2,51.05,51.757207
3,51.80,52.556613
4,51.66,54.813304


In [31]:
pred_lstm = predict_future(model_lstm, 59)

[[0.         0.98773006 0.        ]
 [0.0787234  0.57055215 0.196788  ]
 [0.68510638 1.         0.3723829 ]
 [1.         0.         0.76347328]
 [0.86808511 0.90797546 1.        ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 751ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 624ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 592ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 985ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 614ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step
1/1 ━━━━━

In [33]:
pred_new_data, test_new_data = [], []
for i in range(len(pred_lstm.values)):
    pred_new_data.append(pred_lstm.values[i , 0])
pred_new_data = np.array(pred_new_data)

df = pd.read_csv("E:\Task6\Stock_data_test.csv")
df = df[["close"]]

for i in range(len(df.values)-14):
    test_new_data.append(df.values[i+14 , 0])
test_new_data = np.array(test_new_data)

print(test_new_data.shape)
print(pred_new_data.shape)
test_new_data

<>:6: SyntaxWarning: invalid escape sequence '\T'
<>:6: SyntaxWarning: invalid escape sequence '\T'
C:\Users\DELL inspiron\AppData\Local\Temp\ipykernel_5348\3201630084.py:6: SyntaxWarning: invalid escape sequence '\T'
  df = pd.read_csv("E:\Task6\Stock_data_test.csv")


(59,)
(59,)


array([231.78, 232.15, 235.  , 236.48, 235.86, 230.76, 230.57, 231.41,
       233.4 , 233.67, 230.1 , 225.91, 222.91, 222.01, 223.45, 222.72,
       227.48, 226.96, 224.23, 224.23, 225.12, 228.22, 225.  , 228.02,
       228.28, 229.  , 228.52, 229.87, 232.87, 235.06, 234.93, 237.33,
       239.59, 242.65, 243.01, 243.04, 242.84, 246.75, 247.77, 246.49,
       247.96, 248.13, 251.04, 253.48, 248.05, 249.79, 254.49, 255.27,
       258.2 , 259.02, 255.59, 252.2 , 250.42, 243.85, 243.36, 245.  ,
       242.21, 242.7 , 236.85])

In [34]:
  mse = mean_squared_error(test_new_data, pred_new_data)
  mae = mean_absolute_error(test_new_data, pred_new_data)

  print(f"Mean Absolute Error (MAE): {mae:.4f}")
  print(f"Mean Squared Error (MSE): {mse:.4f}")

Mean Absolute Error (MAE): 9.8601
Mean Squared Error (MSE): 144.7322


In [35]:
accurcy_lstm_newdata = {
    "MAE": mae,
    "MSE": mse
}

In [56]:
with open("apple_model_lstm.pkl", "wb") as f:
    pickle.dump(model_lstm, f)  # save full model

In [39]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Experiment multi model with multi dataset")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

with mlflow.start_run(run_name="model transformer with dataset_test"):
    mlflow.log_params(optimizer_transformer_config)  # بدون أسماء النصوص
    mlflow.log_metrics({"Accurce": accurcy_transformer})  # كقاموس
    mlflow.sklearn.log_model(model_transform, artifact_path="model_transform")

with mlflow.start_run(run_name="model transformer with new dataset"):
    mlflow.log_params(optimizer_transformer_config)
    mlflow.log_metrics({"Accurce": accurcy_transformer_newdata})
    mlflow.sklearn.log_model(model_transform, artifact_path="model_transformer")

with mlflow.start_run(run_name="model lstm with dataset_test"):
    mlflow.log_params(optimizer_lstm_config)
    mlflow.log_metrics({"Accurce": accurcy_lstm})
    mlflow.sklearn.log_model(model_lstm, artifact_path="model_lstm")

with mlflow.start_run(run_name="model lstm with new dataset"):
    mlflow.log_params(optimizer_lstm_config)
    mlflow.log_metrics({"Accurce": accurcy_lstm_newdata})
    mlflow.sklearn.log_model(model_lstm, artifact_path="model_lstm")


MlflowException: API request to http://27.0.0.1:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='27.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Experiment+multi+model+with+multi+dataset (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))

In [1]:
import mlflow
experiment_id = mlflow.create_experiment(
        name="testing_mlflow1",
        artifact_location="testing_mlflow1_artifacts",
        tags={"env": "dev", "version": "1.0.0"},
)

print(experiment_id)

460016626491539729


In [1]:
from mlflow_utils import create_mlflow_experiment

experiment_id = create_mlflow_experiment(experiment_name='testing_mlflow1',
                                        artifact_location='testing_mlflow1_artifacts',
                                        tags={"env": "dev", "version": "1.0.0"})
print(experiment_id)

Experiment testing_mlflow1 already exists.
460016626491539729


In [2]:
from mlflow_utils import get_mlflow_experiment

experiment = get_mlflow_experiment(experiment_id="460016626491539729")

print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
print("Creation timestamp: {}".format(experiment.creation_time))

Name: testing_mlflow1
Experiment_id: 460016626491539729
Artifact Location: file:///e:/Task6/testing_mlflow1_artifacts
Tags: {'env': 'dev', 'version': '1.0.0'}
Lifecycle_stage: active
Creation timestamp: 1736755245241


In [ ]:
mlflow.start_run(run_name="model transformer with dataset_test")
mlflow.log_param("learning_rate",0.0010000000474974513)
mlflow.end_run()

In [4]:
import mlflow
with mlflow.start_run(run_name="model transformer with dataset_test") as run:

    # Your machine learning code goes here
    mlflow.log_param("learning_rate",0.0010000000474974513)
    print("RUN ID")
    print(run.info.run_id)

    print(run.info)

RUN ID
dc9d0343b8d34f079bb22c516af3a8a7
<RunInfo: artifact_uri='file:///e:/Task6/testing_mlflow1_artifacts/dc9d0343b8d34f079bb22c516af3a8a7/artifacts', end_time=None, experiment_id='460016626491539729', lifecycle_stage='active', run_id='dc9d0343b8d34f079bb22c516af3a8a7', run_name='model transformer with dataset_test', run_uuid='dc9d0343b8d34f079bb22c516af3a8a7', start_time=1736756770742, status='RUNNING', user_id='DELL inspiron'>


In [5]:
import mlflow
from mlflow_utils import create_mlflow_experiment, get_mlflow_experiment

experiment_id = create_mlflow_experiment(
    experiment_name="testing_mlflow1",
    artifact_location="testing_mlflow1_artifacts",
    tags={"env": "dev", "version": "1.0.0"},
)
experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))
with mlflow.start_run(run_name="model transformer with dataset_test", experiment_id = experiment.experiment_id) as run:

    # Your machine learning code goes here
    mlflow.log_param("learning_rate",0.0010000000474974513)
    # print run info    
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Experiment testing_mlflow1 already exists.
Name: testing_mlflow1
run_id: 0c07a05acabf4c2c9918b441921e63af
experiment_id: 460016626491539729
status: RUNNING
start_time: 1736757143808
end_time: None
lifecycle_stage: active


In [6]:
import mlflow
from mlflow_utils import get_mlflow_experiment


experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")
print("Name: {}".format(experiment.name))

with mlflow.start_run(run_name="logging_params", experiment_id = experiment.experiment_id) as run:

    # Your machine learning code goes here
    mlflow.log_param("learning_rate", 0.0010000000474974513)

    parameters = {
        "learning_rate": 0.0010000000474974513,
        "epochs": 50,
        "batch_size": 16,
        "loss_function": "mse",
        "optimizer": "adam"
    }

    mlflow.log_params(parameters)

    # print run info
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))



Name: testing_mlflow1
run_id: 069f2acbb5c247518dc37526c4d0bb37
experiment_id: 460016626491539729
status: RUNNING
start_time: 1736757510621
end_time: None
lifecycle_stage: active


In [7]:
import mlflow 
from mlflow_utils import get_mlflow_experiment

experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")
print("Name: {}".format(experiment.name))

with mlflow.start_run(run_name="logging_metrics", experiment_id = experiment.experiment_id) as run:
    # Your machine learning code goes here

    mlflow.log_metric("random_metric", 0.01)

    metrics = {
        "mse": 1.3452,
        "mae": 3.1610,

    }

    mlflow.log_metrics(metrics)

    # print run info
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: 06e8935adccf4860833ac1659c3218b1
experiment_id: 460016626491539729
status: RUNNING
start_time: 1736757712545
end_time: None
lifecycle_stage: active


In [8]:
import mlflow
from mlflow_utils import get_mlflow_experiment

experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")

print("Name: {}".format(experiment.name))

with mlflow.start_run(run_name="logging_artifacts", experiment_id=experiment.experiment_id) as run:

    # your machine learning code goes here

    # create a text file that says hello world
    with open("hello_world.txt", "w") as f:
        f.write("Hello World!")

    # log the text file as an artifact
    mlflow.log_artifact(local_path="hello_world.txt", artifact_path="text_files")

    # print run info
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: 8ce579b28c884e59bc9c641dfd5051fb
experiment_id: 460016626491539729
status: RUNNING
start_time: 1736757765602
end_time: None
lifecycle_stage: active


In [ ]:
experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")
print("Name: {}".format(experiment.name))

with mlflow.start_run(run_name="logging_models", experiment_id=experiment.experiment_id) as run:
    
    # log model using autolog
    mlflow.autolog()

    model_transform.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    verbose=1
)
    result_df, mse, mae = evaluation(X_test, model_transform)

    # print info about the run
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1


2025/01/13 11:59:34 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.7.0, but the installed version is 3.8.0. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
2025/01/13 11:59:34 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/01/13 12:00:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/01/13 12:00:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


Epoch 1/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.0308

91/91 ━━━━━━━━━━━━━━━━━━━━ 91s 266ms/step - loss: 0.0306 - val_loss: 0.0013
Epoch 2/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.0010

91/91 ━━━━━━━━━━━━━━━━━━━━ 19s 205ms/step - loss: 0.0010 - val_loss: 7.9009e-04
Epoch 3/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.0011

...

91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 224ms/step - loss: 7.3264e-04 - val_loss: 6.5928e-05
Epoch 32/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 178ms/step - loss: 5.0203e-04 - val_loss: 7.5466e-05
Epoch 33/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 174ms/step - loss: 6.0402e-04 - val_loss: 4.4238e-04
Epoch 34/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 169ms/step - loss: 5.4298e-04 - val_loss: 6.5386e-04
Epoch 35/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 24s 191ms/step - loss: 6.6114e-04 - val_loss: 6.9756e-05
Epoch 36/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 173ms/step - loss: 5.2378e-04 - val_loss: 2.6531e-04
Epoch 37/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 4.3482e-04

91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 161ms/step - loss: 4.3521e-04 - val_loss: 5.0635e-05
Epoch 38/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 171ms/step - loss: 6.3001e-04 - val_loss: 1.1089e-04
Epoch 39/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 24s 180ms/step - loss: 4.8622e-04 - val_loss: 3.5582e-04
Epoch 40/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 168ms/step - loss: 5.0905e-04 - val_loss: 1.5805e-04
Epoch 41/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 166ms/step - loss: 4.5993e-04 - val_loss: 8.1252e-05
Epoch 42/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 25s 198ms/step - loss: 5.5724e-04 - val_loss: 5.9711e-05
Epoch 43/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 181ms/step - loss: 4.2033e-04 - val_loss: 7.5640e-05
Epoch 44/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 6.2177e-04

91/91 ━━━━━━━━━━━━━━━━━━━━ 18s 193ms/step - loss: 6.2075e-04 - val_loss: 4.6385e-05
Epoch 45/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 23s 197ms/step - loss: 4.2586e-04 - val_loss: 8.6863e-04
Epoch 46/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 168ms/step - loss: 4.5531e-04 - val_loss: 6.4525e-05
Epoch 47/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 16s 166ms/step - loss: 8.1304e-04 - val_loss: 1.7805e-04
Epoch 48/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 17s 177ms/step - loss: 5.9355e-04 - val_loss: 1.4748e-04
Epoch 49/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 21s 171ms/step - loss: 4.8339e-04 - val_loss: 1.2706e-04
Epoch 50/50
91/91 ━━━━━━━━━━━━━━━━━━━━ 22s 232ms/step - loss: 4.8005e-04 - val_loss: 1.0289e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


2025/01/13 12:25:58 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\DELLIN~1\AppData\Local\Temp\tmpzpo8juyt\model, flavor: tensorflow). Fall back to return ['tensorflow==2.18.0', 'cloudpickle==3.1.0']. Set logging level to DEBUG to see the full traceback. 


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 195ms/step
Mean Absolute Error (MAE): 2.2529
Mean Squared Error (MSE): 6.8559
run_id: 140096668cd1477f9ae36a1b72798b9e
experiment_id: 460016626491539729
status: RUNNING
start_time: 1736758770367
end_time: None
lifecycle_stage: active


In [ ]:
if __name__=="__main__":

    run_id = "140096668cd1477f9ae36a1b72798b9e"
        

    # load model
   
    model_uri = 'runs:/140096668cd1477f9ae36a1b72798b9e/model'
    
    transformer = mlflow.tensorflow.load_model(model_uri=model_uri)
    
    print(result_df.head())

c:\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


   Close  Predicted_Close_transform
0  50.39                  56.591337
1  50.39                  55.804379
2  51.05                  55.065600
3  51.80                  55.413070
4  51.66                  55.389325
